# 🤖 Using LLMs with LangChain - A Beginner's Guide

Welcome to this interactive tutorial! In this notebook, you'll learn how to:
- 🔧 Set up and configure chat models
- 💬 Interact with AI using different message types
- 🚀 Make your first API calls to language models

Let's dive in! 🎯

---

## 📚 What are Chat Models?

Chat models are AI systems that:
- Take a **sequence of messages** as input
- Return **intelligent responses** as output

Think of it like having a conversation with an AI assistant!

### 🌟 Why LangChain?

LangChain makes it easy to work with different AI models through a unified interface. In this course, we'll use:

| Model | Why? | Link |
|-------|------|------|
| **ChatOpenAI** | Popular & performant | [Documentation](https://docs.langchain.com/oss/python/integrations/chat/openai) |

> **📝 Note:** You'll need an `OPENAI_API_KEY` to follow along. Don't have one? Check out [OpenAI's platform](https://platform.openai.com/).

---

## 🔑 Step 1: Environment Setup

First, let's load our API keys from the environment. This keeps your secrets safe! 🔒

In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("✅ Environment variables loaded successfully!")

✅ Environment variables loaded successfully!


---

## 🎛️ Step 2: Initialize the Chat Model

Now we'll set up our AI model. We have two options:

### Option A: Using OpenRouter 🔄

OpenRouter allows you to access multiple AI models through a single API.

In [2]:
from langchain_openai import ChatOpenAI
import os

# Configure model with OpenRouter
model = ChatOpenAI(
    model="gpt-4.1-mini",  # Specify the model you want to use
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=0,  # 0 = deterministic, 1 = creative
)

print("🎉 Model initialized with OpenRouter!")

🎉 Model initialized with OpenRouter!


### Option B: Direct OpenAI Connection 🎯

Or connect directly to OpenAI's API:

In [ ]:
from langchain_openai import ChatOpenAI

# Simple direct connection
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

print("🎉 Model initialized with OpenAI!")

🎉 Model initialized with OpenAI!


#### 🌡️ Understanding Temperature

The `temperature` parameter controls response creativity:

```
0.0 ──────────────────────── 1.0
│                            │
Predictable              Creative
Consistent               Varied
Focused                  Exploratory
```

- **temperature=0**: Best for factual answers, code, translations
- **temperature=0.7**: Good balance for most tasks
- **temperature=1**: Maximum creativity for stories, brainstorming

---

## 🚀 Step 3: Your First AI Interaction!

Let's send a message to the AI and see what happens! 🎊

In [4]:
# Sending a simple string message
response = model.invoke("Hello, world!")

print("📩 Full Response Object:")
print("=" * 50)
print(response)
print("=" * 50)

📩 Full Response Object:
content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_7abc656409', 'id': 'chatcmpl-ClR3Y0cnH3PUbeLB2pqKNAc2yp1m3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b0b51-9c81-7871-a689-dc276222920a-0' usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### 🔍 Understanding the Response

The response object contains:
- `content`: The actual AI message
- `response_metadata`: Token usage, costs, model info
- `id`: Unique identifier for this interaction

Most of the time, you'll just want the content:

In [5]:
# Extract just the message content
print("💬 AI says:")
print(response.content)

💬 AI says:
Hello! How can I assist you today?


---

## 💬 Step 4: Working with Message Roles

In real conversations, different participants have different roles. In AI chat, we typically have:

| Role | Purpose | Example |
|------|---------|------|
| **System** 🎭 | Sets behavior/personality | "You are a helpful coding assistant" |
| **User** 👤 | Your questions/inputs | "What is the capital of France?" |
| **Assistant** 🤖 | AI's responses | "The capital of France is Paris." |

![Message Flow](https://via.placeholder.com/700x200/9B59B6/FFFFFF?text=System+Sets+Rules+→+User+Asks+→+Assistant+Answers)

### 🎯 System Messages: Setting the Stage

System messages are like giving the AI a role to play:

In [ ]:
# Creating a structured conversation
response = model.invoke(
    [
        {
            "role": "system",
            "content": "You are a helpful geography expert who loves to share interesting facts.",
        },
        {"role": "user", "content": "What is the capital of France?"},
    ]
)

print("🌍 Geography Expert says:")
print(response.content)

🌍 Geography Expert says:
The capital of France is Paris.


---

## 📨 Step 5: Using Message Objects

LangChain provides special message classes for cleaner code:

```python
HumanMessage    → Messages from the user
AIMessage       → Responses from the AI  
SystemMessage   → System instructions
```

### 🎨 Benefits of Message Objects:
- 📝 More readable code
- 🏷️ Can add metadata (like names)
- 🔄 Easier to build conversation histories

In [ ]:
from langchain_core.messages import HumanMessage

# Create a message with metadata
msg = HumanMessage(
    content="Hello world! I'm excited to learn about AI!",
    name="Student",  # Optional: identify the speaker
)

# Send as a list
messages = [msg]
response = model.invoke(messages)

print("🎓 AI's response to student:")
print(response.content)

🎓 AI's response to student:
Hello! That's awesome to hear! AI is a fascinating field with so much to explore. What would you like to learn about first? Machine learning, natural language processing, computer vision, or something else?


The response contains multiple pieces of information:

```python
AIMessage(
    content='...',              # The actual message text
    response_metadata={         # Metadata about the response
        'token_usage': {...},   # How many tokens were used
        'model_name': '...',    # Which model processed this
        'finish_reason': '...'  # Why the response ended
    },
    id='...',                   # Unique identifier
)
```

### ⚡ Quick Tip: Shorthand Method

For simple queries, you can skip the message objects entirely:

In [8]:
# This automatically converts to a HumanMessage
response = model.invoke("What's the weather like?")
print(response.content)

I don't have access to real-time weather data. You can check the current weather by using a weather website or app like Weather.com, AccuWeather, or a voice assistant on your device. If you tell me your location, I can help guide you on where to find the forecast!


---

## 🛠️ Step 6: Essential Methods

LangChain chat models come with powerful methods:

 📞 `invoke()` - Single Response
```python
response = model.invoke("Your question")
# Returns complete response at once
```

**Best for:** Short answers, quick questions

---

🌊 `stream()` - Real-time Streaming
```python
for chunk in model.stream("Tell me a story"):
    print(chunk.content, end="")
# Shows response as it's generated
```

**Best for:** Long responses, better UX

In [9]:
for chunk in model.stream("Tell me a story"):
    print(chunk.content, end="")

Sure! Here’s a story for you:

---

**The Lantern of Whispering Woods**

In a small village nestled at the edge of the Whispering Woods, there lived a curious girl named Elara. The villagers often spoke of the forest with a mix of fear and wonder, for it was said that the trees could whisper secrets to those who dared to listen.

One evening, as the sun dipped below the horizon, Elara found an old, dusty lantern in her grandmother’s attic. It was unlike any lantern she had seen before—its glass glowed faintly with a soft, blue light, and intricate symbols were etched into its metal frame.

Intrigued, Elara took the lantern and ventured into the Whispering Woods. As she walked deeper among the ancient trees, the whispers grew louder, forming words she could almost understand. The lantern’s light pulsed gently, guiding her path.

Suddenly, she came upon a clearing where the air shimmered with magic. In the center stood a majestic tree, its bark glowing with the same blue light as the lan

Sometimes you might want to use the builtin rompttemplate and the chains of LangChains (But I don't really like them :p)

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

You can use tuple for messages

In [13]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a viral twitter influencer grading a tweet. Generate critique and recommendations for the user's tweet."
            "Always provide detailed recommendations, including requests for length, virality, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Or you can use dicts

In [ ]:
generation_prompt = ChatPromptTemplate.from_messages(
    [
        {
            "role": "system",
            "content": "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
            " Generate the best twitter post possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        },
        {
            "role": "user",
            "content": "I want to write a twitter post about the latest trends in AI.",
        },
    ]
)

In [ ]:
# Using message objects with metadata
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

messages = [
    SystemMessage(content="You are a helpful coding tutor specializing in Python."),
    HumanMessage(
        content="How do I reverse a string in Python?",
        name="Student",  # Optional: identify the speaker
    ),
]

response = model.invoke(messages)

print("🎓 Tutor's Response:")
print(response.content)

🎓 Tutor's Response:
You can reverse a string in Python using slicing. Here's a simple example:

```python
my_string = "Hello, world!"
reversed_string = my_string[::-1]
print(reversed_string)
```

This will output:

```
!dlrow ,olleH
```

The slice `[::-1]` means start from the end towards the first taking each character in reverse order. Let me know if you'd like other methods!


### Building Conversation History 💭

For multi-turn conversations, include previous messages:

In [22]:
# Simulating a multi-turn conversation
conversation = [
    SystemMessage(content="You are a math tutor."),
    HumanMessage(content="What is 15 * 24?"),
    AIMessage(content="15 * 24 = 360"),
    HumanMessage(content="Can you show me how you calculated that?"),
]

response = model.invoke(conversation)

print("📚 Math Tutor:")
print(response.content)

📚 Math Tutor:
Sure! Here's one way to calculate 15 * 24 step-by-step:

1. Break 24 into 20 + 4.
2. Multiply 15 by 20:  
   15 * 20 = 300
3. Multiply 15 by 4:  
   15 * 4 = 60
4. Add the two results together:  
   300 + 60 = 360

So, 15 * 24 = 360.



## LangChain Expression Language (LCEL) 🔗

### What is LCEL? 🤔

LCEL is LangChain's way of chaining components together using the `|` (pipe) operator.

```python
# Traditional approach
formatted = prompt.format(...)
response = model.invoke(formatted)

# LCEL approach ✨
chain = prompt | model
response = chain.invoke({...})
```

### Why Use LCEL? 🌟

| Feature | Benefit |
|---------|--------|
| **Composability** | Chain multiple operations easily |
| **Streaming** | Automatic streaming support |
| **Async** | Built-in async execution |
| **Batching** | Process multiple inputs efficiently |
| **Debugging** | Better error messages and tracing |

### Visual Representation

```
Input ──> Prompt ──> Model ──> Output Parser ──> Final Result
          Template
          
With LCEL:
chain = prompt | model | output_parser
result = chain.invoke(input)
```

In [16]:
llm = ChatOpenAI(model="o4-mini")

generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

In [ ]:
generate_chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="I want to write a twitter post about the latest trends in AI."
            )
        ]
    }
)

AIMessage(content='🔥 AI Trends to Watch in 2024 🔥\n\n• Generative AI (#GPT-4, #StableDiffusion) ignites creativity  \n• Multimodal models bridge vision + language  \n• Self-supervised learning reduces label hunger  \n• Edge AI & tinyML power smarter IoT  \n• Responsible AI & ethics front and center  \n\n#AI #MachineLearning #DeepLearning', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 480, 'prompt_tokens': 66, 'total_tokens': 546, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None, 'id': 'chatcmpl-ClRAuDmcDvPrYDCbBlgAA9xblqOM4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b0b58-8d66-7991-a6b1-56b52b1db585-0', usage_metadata={'input_tokens': 66, 'output_to

In [ ]:
reflect_chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="I want to write a twitter post about the latest trends in AI."
            )
        ]
    }
)

AIMessage(content='Sure—I can help you craft a tweet that grabs attention and drives engagement around AI’s newest trends. To get started:\n\n1. Ask for your draft (so I can critique it)  \n2. Or follow these guidelines and sample template to build your own:\n\n–– Tweet-Writing Best Practices ––  \n• Hook in first 50 characters: pose a provocative question or share a bold stat.  \n• Keep it under 240 characters so it’s easy to RT and quote-tweet.  \n• Use 1–2 high-impact hashtags (#AI #AITrends #MachineLearning).  \n• Tag a key influencer or org if you’re referencing their work.  \n• Add an emoji or two for visual pop, but don’t overdo it.  \n• Ask a follow-up question or invite opinions to boost replies.  \n\n–– Sample Tweet Draft ––  \n“Is AI about to reinvent creativity? 🎨  \nFrom GPT-4’s multimodal art generation to TinyML on your phone, 2024’s AI trends are breaking all the rules.  \nWhich innovation will disrupt your industry next? 👇  \n#AI #AITrends”\n\nCharacter count: ~200  \n

## 📝 Summary

Congratulations! 🎉 You've learned:

✅ How to set up LangChain chat models  
✅ The difference between message roles  
✅ How to invoke models with strings and message objects  
✅ Understanding response objects and extracting content  

### 🚀 Next Steps:

- Learn about prompt templates
- Build conversation chains
- Explore memory and context
- Create RAG agents

### 📚 Resources:

- [LangChain Documentation](https://docs.langchain.com)
- [LangChain Python Reference](https://reference.langchain.com/python)
- [OpenAI API Documentation](https://platform.openai.com/docs)

---

**Happy coding!** 🚀✨